In [ ]:
from qiskit import QuantumCircuit, transpile
import pandas as pd
import numpy as np

from qiskit.circuit import Parameter
from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from tqdm import tqdm


In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
# def min_max_scaling(x,axis=None):
#   """Normalized to [-1, 1]"""
#   min = x.min(axis=axis)
#   max = x.max(axis=axis)
#   result = (x-min)/(max-min)
#   result = (2.0*result)-1.0
#   return result
def min_max_scaling_0_to_2pi(x, axis=None):
    """Normalized to [0, 2π]"""
    min_val = x.min(axis=axis)
    max_val = x.max(axis=axis)
    result = (x - min_val) / (max_val - min_val)  
    result = 2.0 * np.pi * result  
    return result

In [ ]:
raw_X = df.drop(columns=['Label'])
features = min_max_scaling_0_to_2pi(raw_X.values)
Y = df['Label']
labels = Y.values

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    features,
    labels,
    test_size=0.2, 
    random_state=42,  
    stratify=labels  
)

# Quantum Support Vector Machine

In [ ]:
def kernelcircuit(x,y):
    qc = QuantumCircuit(3)
    feature_map = ZZFeatureMap(3,reps=1)
    feature_map = feature_map.assign_parameters(x)
    qc = qc.compose(feature_map)
    qc.barrier()
    feature_map2 = ZZFeatureMap(3,reps=1)
    feature_map2 = feature_map2.assign_parameters(y)
    feature_map_inv=feature_map2.inverse()
    qc = qc.compose(feature_map_inv)
    qc.measure_all()
    simulator = AerSimulator()
    transpiled = transpile(qc, simulator)
    job = simulator.run(transpiled, shots=256)
    result = job.result()
    counts = result.get_counts()
    print(counts)
    zero_state = '0' * 3
    return counts.get(zero_state, 0) / 256

In [ ]:
n_samples = len(X_train)
kernel_matrix = np.zeros((n_samples, n_samples))

with tqdm(total=n_samples*(n_samples+1)/2, desc="Building Kernel Matrix") as pbar:
    for i in range(n_samples):
        for j in range(i, n_samples):
            kernel_matrix[i, j] = kernelcircuit(X_train[i], X_train[j])
            kernel_matrix[j, i] = kernel_matrix[i, j]  # Symmetric
            pbar.update(1)


In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='precomputed')
svm_model.fit(kernel_matrix, y_train)

n_test = len(X_test)
n_train = len(X_train)
kernel_test = np.zeros((n_test, n_train))

with tqdm(total=n_test * n_train, desc="Building Test Kernel Matrix") as pbar:
    for i in range(n_test):
        for j in range(n_train):
            kernel_test[i, j] = kernelcircuit(X_test[i], X_train[j])
            pbar.update(1)

y_pred = svm_model.predict(kernel_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import (classification_report, confusion_matrix,
                             precision_recall_curve, roc_curve, roc_auc_score)

print("Classification Report:")
print(classification_report(y_test, y_pred))

if len(np.unique(y_train)) == 2:
    fpr, tpr, thresholds = roc_curve(y_test, svm_model.decision_function(kernel_test))
    roc_auc = roc_auc_score(y_test, y_pred)

    precision, recall, _ = precision_recall_curve(y_test, svm_model.decision_function(kernel_test))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=np.unique(y_train),
            yticklabels=np.unique(y_train))
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()